Libraries

In [4]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import re

import utils as lf

In [5]:
lf.get_table('2liga')

,Poz,Zespół,Mecze_rozegrane,Pkt.,Pkt.,Z,R,P,BZ,BS,+/-,Forma
0,1,Husaria Mokotów,8,1,21,7,0,1,70,20,50,"W,W,P,W,W"
1,2,Dziki Młochów,8,2,18,6,0,2,53,35,18,"W,W,W,P,W"
2,3,UEFA Mafia Ursynów,8,3,15,5,0,3,48,38,10,"W,W,W,W,W"
3,4,KSB Warszawa,8,4,15,5,0,3,68,49,19,"W,W,P,P,P"
4,5,Orzeły Stolicy,8,5,13,4,1,3,33,26,7,"P,P,P,W,W"
5,6,AFC Goodfellas,8,6,10,3,1,4,37,47,-10,"W,W,P,W,P"
6,7,Green Lantern,8,7,9,3,0,5,43,58,-15,"P,P,W,P,W"
7,8,Tylko Zwycięstwo,8,8,9,3,0,5,28,47,-19,"P,P,W,W,P"
8,9,Warszawska Ferajna,8,9,6,2,0,6,22,61,-39,"P,P,W,P,P"
9,10,Korsarze,8,10,2,1,0,7,27,48,-21,"P,P,P,P,P"


In [6]:
# Index improvement needed
lf.get_matches('2liga', round_=7)

,Kol.,Data,Godz.,Gospodarz,,Gość,Video,,Sędzia,Boisko,Raport
2,7,2023-11-12,09:00,Dziki Młochów,7 : 4,Tylko Zwycięstwo,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,14:00,Orzeły Stolicy,3 : 5,Husaria Mokotów,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,17:00,Korsarze,3 : 8,AFC Goodfellas,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,19:00,UEFA Mafia Ursynów,8 : 2,Warszawska Ferajna,,,Piotr Krajczyński,Arena AWF,Raport
2,7,2023-11-12,19:00,Green Lantern,6 : 18,KSB Warszawa,,,Rafał Szczytniewski,Arena AWF,Raport


# Working area

In [113]:
# Linki do wyników

In [114]:
dff = table_of_links(url)

In [118]:
def reports_links(url):
    list_ = extract_mecze_details_links(extract_mecze_links(url)) #dff.Zespół[0])
    list_ = ['https://ligafanow.pl/'+ elem for elem in list_]
    return list_

In [120]:
b = reports_links(dff.Zespół[0])
b

['https://ligafanow.pl//statystyki/raport/8058',
 'https://ligafanow.pl//statystyki/raport/7989',
 'https://ligafanow.pl//statystyki/raport/7919',
 'https://ligafanow.pl//statystyki/raport/8198',
 'https://ligafanow.pl//statystyki/raport/8272',
 'https://ligafanow.pl//statystyki/raport/7754',
 'https://ligafanow.pl//statystyki/raport/7844',
 'https://ligafanow.pl//statystyki/raport/8130',
 'https://ligafanow.pl//statystyki/raport/7683']

In [121]:
def match_details(path):
    response = requests.get(path)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Znajdź wszystkie tabelki na stronie
    tables = soup.find_all('table')

    # Przechowaj ramki danych w liście
    dataframes = []

    # Sprawdź, czy znaleziono jakiekolwiek tabele
    if tables:
        # Iteruj przez wszystkie znalezione tabele
        for table in tables:
            # Wydziel dane z tabeli do listy słowników
            table_data = []
            rows = table.find_all('tr')

            # Sprawdź, czy istnieje przynajmniej jeden wiersz
            if rows:
                header = [header.text.strip() for header in rows[0].find_all(['th', 'td'])]

                # Iteruj przez pozostałe wiersze
                for row in rows[1:]:
                    row_data = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
                    table_data.append(dict(zip(header, row_data)))

                # Przekształć dane do DataFrame
                df = pd.DataFrame(table_data)
                dataframes.append(df)

    # Wydrukuj ramki danych
    df = pd.concat([dataframes[1],dataframes[2]]).dropna()
    
    return df

In [122]:
match_details(b[0])

,,Imie i nazwisko,Wydarzenie,Ilość
1,,Maksym Popov,Bramka stracona,1
3,,Igor Polskyi,Bramka,2
4,,Illia Kopyl,Bramka,1
5,,Vladyslav Barabash,Bramka,2
7,,Anrdei Benenko,Asysta,2
8,,Illia Kopyl,Asysta,1
9,,Vladyslav Barabash,Asysta,1
11,,Vladyslav Barabash,SuperStar,3
13,,Maksym Popov,6 - kolejki,1
1,,Artur Pręgowski,Bramka stracona,5


In [123]:
# Funkcja do ekstrakcji danych z tabeli
def extract_table_data(table):
    table_data = []
    
    # Znajdź nagłówki z pierwszego wiersza thead
    header_row = table.select('thead tr')[1]  # Use the second row to get headers with tooltip
    headers = []

    for header in header_row.find_all(['th', 'td']):
        if 'tooltip' in header.attrs:
            headers.append(header['tooltip'].strip())
        else:
            headers.append(header.text.strip())

    rows = table.select('tbody tr')
    
    for row in rows:
        player_data = {}
        columns = row.find_all(['td', 'th'])
        
        # Iteruj przez kolumny w danym wierszu
        for i, col in enumerate(columns):
            player_data[headers[i]] = col.text.strip()
        
        table_data.append(player_data)

    return table_data

def extract_team(url):
    response = requests.get(url)
    soup = BeautifulSoup(response1.text, 'html.parser')
    
    table1 = soup.find('table', {'id': 'mytxablecc'})
    table2 = soup.find('table', {'id': 'mytxablec'})
    
    # Sprawdź, czy obie tabele zostały znalezione
    if table1 and table2:
        # Ekstrahuj dane z obu tabel
        data1 = extract_table_data(table1)
        data2 = extract_table_data(table2)
    
    # Połącz dane z obu tabel w jedną listę
    combined_data = data1 + data2
    
    # Przekształć dane do DataFrame
    df = pd.DataFrame(combined_data)
    df.columns = ['Imie i nazwisko', 'Numer', 'Liczba wystepów', 'Liczba bramek',
       'Asysty', 'Kanadyjcztk', 'Superstar', 'Top6', 'MVP', 'Czerwone kartki',
       'Zółte kartki', 'Stracone bramki', 'Samobój', 'Obronione karne',
       'Czyste konto', 'Gold Team','ID']
    return df

In [124]:
extract_team(dff.Zespół[0])

,Imie i nazwisko,Numer,Liczba wystepów,Liczba bramek,Asysty,Kanadyjcztk,Superstar,Top6,MVP,Czerwone kartki,Zółte kartki,Stracone bramki,Samobój,Obronione karne,Czyste konto,Gold Team,ID
0,Jędrzej Skawina,,4,1,0,1,0,0,0,0,0,21,0,0,0,0,8098
1,Artur Pręgowski,,4,0,0,0,0,0,0,0,1,16,0,0,0,0,6970
2,Dominik Sposób,,1,0,0,0,1,0,0,0,0,4,0,0,0,0,9016
3,Janek Grzybowski,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9015
4,Misha Lishtwan,,4,2,1,3,0,0,0,0,0,0,0,0,0,0,8983
5,Aleksander Popecki,,2,0,2,2,1,0,0,0,0,0,0,0,0,0,8969
6,Piotr Lewit,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8967
7,Jura Lunio,,5,3,2,5,3,1,0,0,0,0,0,0,0,0,8946
8,Bartosz Krzeszowski,,2,3,0,3,0,0,0,0,0,0,0,0,0,0,8945
9,Piotr Ignatowski,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8424


In [1]:
# Strzelcy

---------------------------------------------------------------------

In [25]:
# Pola do zrobienia z nich tabel

In [123]:
df.Forma[2]

'W 2023-11-12 21:00    In Plus & Pojemna Halina    5 : 7  Gladiatorzy Eternis    Arena AWF  W 2023-11-05 19:00    Gladiatorzy Eternis   14 : 2    ALPAN     Arena AWFR 2023-10-29 18:00    EXC Mobile Ochota   4 : 4  Gladiatorzy Eternis    Arena AWF  W 2023-11-26 18:00    TUR Ochota    1 : 3  Gladiatorzy Eternis    Arena AWF  W 2023-10-15 20:00    Gladiatorzy Eternis   14 : 4    Esportivo Varsovia     Arena AWF'

In [124]:
# zapisanie zmiennej tekstowej do zrobienia z niej tabeli
data = df.Forma[2]

In [125]:
# Prawdziwy kod

In [126]:
# Podzielenie danych na wiersze
rows = re.split(r'([PWR])\s+(\d{4}-\d{2}-\d{2} \d{2}:\d{2})', data)[1:]

In [129]:

# Utwórz DataFrame
columns = ["Typ", "Data", "Czas", "Drużyna 1", "Wynik", "Drużyna 2", "Arena"]
df = pd.DataFrame([row.split()[0:3] + [rozdziel_na_podteksty(row)[1]] + [rozdziel_na_podteksty(row)[2]]+ [rozdziel_na_podteksty(row)[3]] + [rozdziel_na_podteksty(row)[4]] for row in triple_strings(rows)], columns=columns)

In [131]:
df

,Typ,Data,Czas,Drużyna 1,Wynik,Drużyna 2,Arena
0,W,2023-11-12,21:00,In Plus & Pojemna Halina,5 : 7,Gladiatorzy Eternis,Arena AWF
1,W,2023-11-05,19:00,Gladiatorzy Eternis,14 : 2,ALPAN,Arena AWF
2,R,2023-10-29,18:00,EXC Mobile Ochota,4 : 4,Gladiatorzy Eternis,Arena AWF
3,W,2023-11-26,18:00,TUR Ochota,1 : 3,Gladiatorzy Eternis,Arena AWF
4,W,2023-10-15,20:00,Gladiatorzy Eternis,14 : 4,Esportivo Varsovia,Arena AWF
